In [ ]:
# https://wikidocs.net/60691

# 02. IMDB 리뷰 감성 분류하기(IMDB Movie Review Sentiment Analysis)

참고 논문: http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

In [1]:
# 영화 사이트 IMDB의 리뷰 데이터를 이용해 1 긍정 , 0 부정을 분류해보자

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import random

In [3]:
# 랜덤시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [13]:
# 하이퍼파라미터 변수
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [39]:
# 디바이스 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cpu


In [38]:
# GPU availability 확인
!nvidia-smi3

'nvidia-smi3'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [37]:
import torch

# 현재 Setup 되어있는 device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())

print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  0


AssertionError: Torch not compiled with CUDA enabled

In [40]:
import torch
torch.cuda.is_available()

False

In [41]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Aug_15_21:18:57_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.4, V11.4.120
Build cuda_11.4.r11.4/compiler.30300941_0


In [42]:
!pip list

Package                 Version
----------------------- -------------------
absl-py                 0.13.0
astunparse              1.6.3
backcall                0.2.0
beautifulsoup4          4.6.0
blis                    0.7.4
cachetools              4.2.2
catalogue               2.0.6
certifi                 2021.5.30
charset-normalizer      2.0.6
click                   8.0.1
colorama                0.4.4
cycler                  0.10.0
cymem                   2.0.5
Cython                  0.29.14
debugpy                 1.4.1
decorator               5.0.9
en-core-web-sm          3.1.0
en-core-web-trf         3.1.0
entrypoints             0.3
eunjeon                 0.4.0
filelock                3.0.12
flatbuffers             1.12
gast                    0.3.3
gensim                  3.8.3
glove-python-binary     0.2.0
google-auth             1.35.0
google-auth-oauthlib    0.4.6
google-pasta            0.2.0
grpcio                  1.32.0
h5py                    2.10.0
huggingface-hub

In [30]:
!python -V

Python 3.8.5


In [33]:
print(torch.version.cuda)

None


In [34]:
!conda install import torch
print(torch.cuda.get_device_name(0)
print(torch.cuda.is_available())
print(torch.__version__) torchvision cudatoolkit=11.4 -c pytorch

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - cudatoolkit=11.4

Current channels:

  - https://conda.anaconda.org/pytorch/win-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [36]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.__version__)

AssertionError: Torch not compiled with CUDA enabled